## Installation - Libraries

In [1]:
!pip3 install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0rc2 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.11.0 which is incompatible.
farm-haystack 1.14.0 requires protobuf<=3.20.2, but you have protobuf 4.23.3 which is incompatible.


In [5]:
!pip3 install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 6.1 MB/s eta 0:00:0000:01


In [135]:
import os
import pandas as pd
import pickle
import re
import json
from groq import Groq

In [6]:
client = Groq(api_key="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")


## Series of Prompts 

In [7]:

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "You are an AI-powered troubleshooting agent tailored for IT service management (ITSM), capable of autonomously performing troubleshooting tasks using integrated IT tools and systems. Considering the examples provided below. Please provide a JSON output for similar incidents provided",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

I'm ready to help! Please provide the examples of incidents you'd like me to troubleshoot, and I'll generate a JSON output for each incident. Please format the examples in a concise and standardized way, including relevant information such as:

* Incident ID
* Date/Time
* Device/IP Address
* Error Message/ Symptoms
* Last Known Good Configuration (if applicable)

Once you provide the examples, I'll use my ITSM capabilities to analyze the incidents and generate a JSON output containing the troubleshooting results, recommendations, and potential solutions.


In [8]:
example_data = """[{ "context": "A user reports that their laptop is overheating frequently, causing automatic shutdowns.","metadata": {"operating_system": "Windows 10",
 "available_tools": ["System Center Operations Manage
r", "Power Management Software", "BIOS Update Software", "A
tomicwork"]
 },
 "tasks": [
 {
 "description": "Run a system diagnostics using Syst
em Center Operations Manager to check for high CPU usage or
potential hardware issues causing overheating.",
 "tools": ["System Center Operations Manager"]
 },
 { "description": "Automatically adjust the power sett
ings using Power Management Software to manage overheating
issues.",
 "tools": ["Power Management Software"]
 },
 {
 "description": "Update the system BIOS if an outdat
ed version is detected to improve hardware management.",
 "tools": ["BIOS Update Software"]
 },
 {
 "description": "Schedule a maintenance check if the
problem persists after automated adjustments.",
 "tools": ["Atomicwork"]
 }
 ]
 },
 {
 "context": "Several users are experiencing slow interne
t connectivity in the marketing department.",
 "metadata": {
 "operating_system": "Windows 10",
 "available_tools": ["Network Performance Monitoring S
oftware", "Network Management System", "Device Update Manag
ement Software", "Atomicwork"] },
 "tasks": [
 {
 "description": "Automatically test network speed an
d diagnose connectivity issues using network management too
ls.",
 "tools": ["Network Performance Monitoring Softwar
e"]
 },
 {
 "description": "Reset the network routers and switc
hes remotely to resolve common connectivity issues.",
 "tools": ["Network Management System"] },
 {
 "description": "Check for and apply firmware update
s to network devices to ensure optimal performance.",
 "tools": ["Device Update Management Software"]
 },
 {
 "description": "Report to the IT team if issues are
unresolved, with detailed diagnostics and recommended next
steps.",
 "tools": ["Atomicwork"]
 }
 ]
 },
 {
 "context": "The main office printer is not responding t
o print commands.",
 "metadata": {
 "operating_system": "Windows 10",
 "available_tools": ["Printer Management Software", "P
rint Job Management Tool", "Hardware Diagnostic Tools", "At
omicwork"]
 },
 "tasks": [
 {
 "description": "Check the printer's online status a
nd attempt a remote reset to resolve common errors.",
 "tools": ["Printer Management Software" ]
 },
 {
 "description": "Automatically clear the print queue
if a jam is detected and restart the printing process.",
 "tools": ["Print Job Management Tool"]
 },
 {
 "description": "Run a diagnostic to check for hardw
are faults and automatically send a report to the maintenan
ce team if necessary.", "tools": ["Hardware Diagnostic Tools"]
 },
 {
 "description": "Notify the administrative staff thr
ough the IT management system if the printer needs physical
inspection.",
 "tools": ["Atomicwork"]
 }
 ]
 }
] - These are examples  of few scenarios that highlight different types of IT issues, from
network problems to software malfunctions, each with a specified
sequence of tasks and tools for resolution."""

In [9]:
ex = """{
 "context": "The main office printer is not responding t
o print commands.",
 "metadata": {
 "operating_system": "Windows 10",
 "available_tools": ["Printer Management Software", "P
rint Job Management Tool", "Hardware Diagnostic Tools", "At
omicwork"]
 },
 "tasks": [
 {
 "description": "Check the printer's online status a
nd attempt a remote reset to resolve common errors.",
 "tools": ["Printer Management Software" ]
 },
 {
 "description": "Automatically clear the print queue
if a jam is detected and restart the printing process.",
 "tools": ["Print Job Management Tool"]
 },
 {
 "description": "Run a diagnostic to check for hardw
are faults and automatically send a report to the maintenan
ce team if necessary.", "tools": ["Hardware Diagnostic Tools"]
 },
 {
 "description": "Notify the administrative staff thr
ough the IT management system if the printer needs physical
inspection.",
 "tools": ["Atomicwork"]
 }
 ]
 }"""

In [10]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "First I am giving you examples as to how I want the data to look. These are some examples: "+ example_data
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

I have taken note of the examples you have provided. It appears that each example represents a specific IT issue with a corresponding set of tasks and tools necessary to resolve the issue. The data structure for each example seems to follow a consistent pattern.

For instance, each example consists of a "context" field that describes the IT issue, a "metadata" field that provides additional context, and a "tasks" field that contains a list of tasks to resolve the issue. Each task includes a "description" field that describes the task and a "tools" field that specifies the tools or software required to complete the task.

Please confirm if my understanding of your examples is correct. Also, please let me know how you would like me to assist you further. Would you like me to generate similar examples or data based on your provided examples?


## ITSM DataSet - Kaggle

In [11]:
df = pd.read_excel("Incidents_Test_Database.xlsx")

In [12]:
incidents = list(df["description"])


In [13]:
def give_output(data):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Generate data in the same format as given in " + ex + "Generate the same for this incident. Give a list of tasks (more than one) user can perform. Always add tools in each tasks. OS in the metadata should be always given. Add atomicwork tool also in some cases when physical help is required or ticket needs to be raised. Do not add ... in the end of text" + data
        }
    ],
    model="llama3-8b-8192",
        )

    return chat_completion.choices[0].message.content
    

In [15]:
l = give_output(incidents[0])

In [16]:
print(l)

Here is the generated data:

{
  "context": "Printer failed to print the document",
  "metadata": {
    "operating_system": "Windows 10",
    "available_tools": ["Printer Management Software", "Print Job Management Tool", "Hardware Diagnostic Tools", "Atomicwork"]
  },
  "tasks": [
    {
      "description": "Check the printer's settings and ensure the print job is not stuck in the queue.",
      "tools": ["Printer Management Software"]
    },
    {
      "description": "Clear the print queue and restart the printing process.",
      "tools": ["Print Job Management Tool"]
    },
    {
      "description": "Run a diagnostic to check for any hardware faults and automatically send a report to the maintenance team if necessary.",
      "tools": ["Hardware Diagnostic Tools"]
    },
    {
      "description": "If the issue persists, initiate a ticket support request to the printer vendor for further assistance.",
      "tools": ["Atomicwork"]
    },
    {
      "description": "Verify that th

## We made first 100 calls first day to LLM and then second 100 the next day - (to maintain daily token usage limit)

In [58]:
#first 100 calls to LLM
# lst = []
# for i in range(0,100):
#     temp_lst = give_output(incidents[i])
#     lst.append(temp_lst)

In [96]:
lst = []
for i in range(100,200):
    temp_lst = give_output(incidents[i])
    lst.append(temp_lst)



In [97]:
for i in range(len(lst)):
    lst[i] = lst[i].replace("Here is the generated data:\n\n","")

In [99]:
print(lst[9])

Here is the generated data in the same format:

{
 "context": "Can't login in the AVD, in production",
 "metadata": {
  "operating_system": "Windows 10",
  "available_tools": ["Remote Desktop Connection", "AVD Manager", "Virtual Device Configuration Tool", "Atomicwork"]
 },
 "tasks": [
  {
   "description": "Check the AVD configuration and restart the Virtual Device Manager to ensure all services are running correctly.",
   "tools": ["Remote Desktop Connection", "AVD Manager"]
  },
  {
   "description": "Verify the login credentials and ensure they are correct, then try to login again.",
   "tools": ["Remote Desktop Connection"]
  },
  {
   "description": "Check for any potential conflicts with other applications or services running on the same network and try to troubleshoot the issue.",
   "tools": ["Virtual Device Configuration Tool"]
  },
  {
   "description": "Raise a ticket to the IT team to assist with resolving the issue or require physical assistance from the infrastructure te

## Storing intermediate data in pickle file

In [83]:
# #first 100
# import pickle
# with open('parrot.pkl', 'wb') as f:
#     pickle.dump(lst, f)

In [100]:
#second 100
with open('parrot2.pkl', 'wb') as f:
    pickle.dump(lst, f)

In [113]:
with open('parrot.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [114]:
#second 100
with open('parrot2.pkl', 'rb') as f:
    mynewlist2 = pickle.load(f)

In [115]:
mynewlist 


['{\n  "context": "Printer failed to print the document",\n  "metadata": {\n    "operating_system": "Windows 10"\n  },\n  "tasks": [\n    {\n      "description": "Check for any paper jams or misaligned printing and restart the print job.",\n      "tools": ["Printer Management Software", "Atomicwork"]\n    },\n    {\n      "description": "Configure the document properties to ensure the correct print settings are selected.",\n      "tools": ["Printer Management Software"]\n    },\n    {\n      "description": "Verify that the printer is properly connected to the network and is receiving the print job.",\n      "tools": ["Print Job Management Tool", "Atomicwork"]\n    },\n    {\n      "description": "If the issue persists, raise a ticket with the IT team to investigate and resolve the issue.",\n      "tools": ["Atomicwork"]\n    }\n  ]\n}\n\nLet me know if you\'d like me to generate more data!',
 '{\n "context": "My headset stopped working in the left side",\n "metadata": {\n "operating_sy

In [116]:
mynewlist2

['{\n  "context": "Tried to Login in SAP without success",\n  "metadata": {\n    "operating_system": "Windows 10",\n  },\n  "tasks": [\n    {\n      "description": "Verify user credentials and attempt to log in again.",\n      "tools": ["Atomicwork"]\n    },\n    {\n      "description": "Check the system clock and ensure it is synchronized with the network.",\n      "tools": ["Atomicwork"]\n    },\n    {\n      "description": "Check the SAP login configuration files for any discrepancies.",\n      "tools": []\n    },\n    {\n      "description": "Reach out to the SAP admin team for assistance.",\n      "tools": ["Atomicwork"]\n    }\n  ]\n}\n\nLet me know if you want me to generate another one!',
 'Here is the generated data in the same format:\n\n{\n "context": "Tried to Login in SAP without success",\n "metadata": {\n "operating_system": "Windows 10",\n "available_tools": ["SAP login manager", "SAP error log viewer", "Atomicwork", "SAP system admin tool"]\n },\n "tasks": [\n {\n "des

In [117]:
mynewlist +=mynewlist2

In [118]:
len(mynewlist)

200

In [24]:
mynewlist

['{\n  "context": "Printer failed to print the document",\n  "metadata": {\n    "operating_system": "Windows 10"\n  },\n  "tasks": [\n    {\n      "description": "Check for any paper jams or misaligned printing and restart the print job.",\n      "tools": ["Printer Management Software", "Atomicwork"]\n    },\n    {\n      "description": "Configure the document properties to ensure the correct print settings are selected.",\n      "tools": ["Printer Management Software"]\n    },\n    {\n      "description": "Verify that the printer is properly connected to the network and is receiving the print job.",\n      "tools": ["Print Job Management Tool", "Atomicwork"]\n    },\n    {\n      "description": "If the issue persists, raise a ticket with the IT team to investigate and resolve the issue.",\n      "tools": ["Atomicwork"]\n    }\n  ]\n}\n\nLet me know if you\'d like me to generate more data!',
 '{\n "context": "My headset stopped working in the left side",\n "metadata": {\n "operating_sy

## Extracting context, metadata and tasks from the json string - using regex

In [129]:


def extract_tasks_with_regex(input_string):
    # Define the regex pattern to match task items
    task_item_pattern = r'{\s*"description":\s*"([^"]+)",\s*"tools":\s*\[([^\]]+)\]\s*}'

    # Find all matches for task items in the input string
    task_matches = re.findall(task_item_pattern, input_string)

    # Extract descriptions and tools from each match
    tasks_list = []
    for match in task_matches:
        description = match[0]
        tools = [tool.strip('" ') for tool in match[1].split(',')]
        tasks_list.append({"description": description, "tools": tools})

    return tasks_list

In [136]:
def get_individual_components(input_string):

    # Regex patterns to extract context, metadata, and tasks
    context_pattern = r'"context": "(.*?)"'
    metadata_pattern = r'"metadata": {(.*?)}'
    # tasks_pattern = r'"tasks": \[(.*?)\]'
    
    # Extracting components using regex
    context_match = re.search(context_pattern, input_string, re.DOTALL)
    metadata_match = re.search(metadata_pattern, input_string, re.DOTALL)
    # tasks_match = re.search(tasks_pattern, input_string, re.DOTALL)

    # Printing extracted components
    st = "\""
    context = st+context_match.group(1).strip()+st
    metadata = metadata_match.group(1).strip()
    tasks = json.dumps(extract_tasks_with_regex(input_string))
    # if context_match:
    #     print("Context:", st+context_match.group(1).strip()+st)

    # if metadata_match:
    #     print("Metadata:", metadata_match.group(1).strip())

    # if tasks_match:
    #     print("Tasks:", tasks_match.group(1).strip()+"]}")
    return context,metadata,tasks


In [138]:
dic = {"Context":[],"Metadata":[],"Tasks":[]}
for i in range(len(mynewlist)):
    context,metadata,tasks = get_individual_components(mynewlist[i])
    dic["Context"].append(context.replace("\n"," "))
    dic["Metadata"].append(metadata.replace("\n"," "))
    dic["Tasks"].append(tasks.replace("\n"," "))

In [139]:
dic

{'Context': ['"Printer failed to print the document"',
  '"My headset stopped working in the left side"',
  '"Mouse stopped working, need replacement"',
  '"Cannot log in to SAP system."',
  '"Tried to Login in SAP without success"',
  '"Printer EB065 printing with bad quality"',
  '"Production Machine is not accessible remotely"',
  '"Can\'t download the PDF report from SAP"',
  '"PDF is downloaded with failure, can\'t open the file"',
  '"Cannot access the cloud service."',
  '"Tried to print but the document stayed in the queue"',
  '"Can\'t login in the AVD, in production"',
  '"New password setup failed in SAP"',
  '"Barcode reader not working in the final inspection."',
  '"AVD with low performance"',
  '"Monitor not showing the screen"',
  '"Computer does not turn on"',
  '"Cannot download the Excel file from the system."',
  '"Remote Access Issue: Unable to connect to the remote desktop."',
  '"Tried to Login in SAP without success"',
  '"Printer failed to print the document"',

## Adding the three examples in the assignment document to the list

In [140]:
example_data_formatted = [
    """{
        "context": "A user reports that their laptop is overheating frequently, causing automatic shutdowns.",
        "metadata": {
            "operating_system": "Windows 10",
            "available_tools": [
                "System Center Operations Manager",
                "Power Management Software",
                "BIOS Update Software",
                "Atomicwork"
            ],
        },
        "tasks": [
            {
                "description": "Run a system diagnostics using System Center Operations Manager to check for high CPU usage or potential hardware issues causing overheating.",
                "tools": ["System Center Operations Manager"],
            },
            {
                "description": "Automatically adjust the power settings using Power Management Software to manage overheatingissues.",
                "tools": ["Power Management Software"],
            },
            {
                "description": "Update the system BIOS if an outdated version is detected to improve hardware management.",
                "tools": ["BIOS Update Software"],
            },
            {
                "description": "Schedule a maintenance check if theproblem persists after automated adjustments.",
                "tools": ["Atomicwork"]
            },
        ],
    }""",
    """{
        "context": "Several users are experiencing slow internet connectivity in the marketing department.",
        "metadata": {
            "operating_system": "Windows 10",
            "available_tools": [
                "Network Performance Monitoring Software",
                "Network Management System",
                "Device Update Management Software",
                "Atomicwork"
            ],
        },
        "tasks": [
            {
                "description": "Automatically test network speed and diagnose connectivity issues using network management tools.",
                "tools": ["Network Performance Monitoring Software"],
            },
            {
                "description": "Reset the network routers and switches remotely to resolve common connectivity issues.",
                "tools": ["Network Management System"],
            },
            {
                "description": "Check for and apply firmware updates to network devices to ensure optimal performance.",
                "tools": ["Device Update Management Software"],
            },
            {
                "description": "Report to the IT team if issues are unresolved, with detailed diagnostics and recommended next steps.",
                "tools": ["Atomicwork"]
            },
        ],
    }""",
    """{
        "context": "The main office printer is not responding to print commands.",
        "metadata": {
            "operating_system": "Windows 10",
            "available_tools": [
                "Printer Management Software",
                "Print Job Management Tool",
                "Hardware Diagnostic Tools",
                "Atomicwork"
            ],
        },
        "tasks": [
            {
                "description": "Check the printer's online status and attempt a remote reset to resolve common errors.",
                "tools": ["Printer Management Software"],
            },
            {
                "description": "Automatically clear the print queue if a jam is detected and restart the printing process.",
                "tools": ["Print Job Management Tool"],
            },
            {
                "description": "Run a diagnostic to check for hardware faults and automatically send a report to the maintenance team if necessary.",
                "tools": ["Hardware Diagnostic Tools"],
            },
            {
                "description": "Notify the administrative staff through the IT management system if the printer needs physical inspection.",
                "tools": ["Atomicwork"]
            },
        ],
    }"""
]


 

In [141]:
for exmpl in example_data_formatted:
    context,metadata,tasks = get_individual_components(mynewlist[i])
    dic["Context"].append(context.replace("\n"," "))
    metadata = metadata.replace("\n"," ")
    metadata = metadata.replace("  "," ")
    dic["Metadata"].append(metadata)
    dic["Tasks"].append(tasks.replace("\n"," "))
   

In [142]:
dic

{'Context': ['"Printer failed to print the document"',
  '"My headset stopped working in the left side"',
  '"Mouse stopped working, need replacement"',
  '"Cannot log in to SAP system."',
  '"Tried to Login in SAP without success"',
  '"Printer EB065 printing with bad quality"',
  '"Production Machine is not accessible remotely"',
  '"Can\'t download the PDF report from SAP"',
  '"PDF is downloaded with failure, can\'t open the file"',
  '"Cannot access the cloud service."',
  '"Tried to print but the document stayed in the queue"',
  '"Can\'t login in the AVD, in production"',
  '"New password setup failed in SAP"',
  '"Barcode reader not working in the final inspection."',
  '"AVD with low performance"',
  '"Monitor not showing the screen"',
  '"Computer does not turn on"',
  '"Cannot download the Excel file from the system."',
  '"Remote Access Issue: Unable to connect to the remote desktop."',
  '"Tried to Login in SAP without success"',
  '"Printer failed to print the document"',

In [143]:
dic

{'Context': ['"Printer failed to print the document"',
  '"My headset stopped working in the left side"',
  '"Mouse stopped working, need replacement"',
  '"Cannot log in to SAP system."',
  '"Tried to Login in SAP without success"',
  '"Printer EB065 printing with bad quality"',
  '"Production Machine is not accessible remotely"',
  '"Can\'t download the PDF report from SAP"',
  '"PDF is downloaded with failure, can\'t open the file"',
  '"Cannot access the cloud service."',
  '"Tried to print but the document stayed in the queue"',
  '"Can\'t login in the AVD, in production"',
  '"New password setup failed in SAP"',
  '"Barcode reader not working in the final inspection."',
  '"AVD with low performance"',
  '"Monitor not showing the screen"',
  '"Computer does not turn on"',
  '"Cannot download the Excel file from the system."',
  '"Remote Access Issue: Unable to connect to the remote desktop."',
  '"Tried to Login in SAP without success"',
  '"Printer failed to print the document"',

## Creating intermediate Dataframe 

In [144]:
df_new = pd.DataFrame(dic)

In [145]:
df_new

,Context,Metadata,Tasks
0,"""Printer failed to print the document""","""operating_system"": ""Windows 10""","[{""description"": ""Check for any paper jams or ..."
1,"""My headset stopped working in the left side""","""operating_system"": ""Windows 10"", ""available_...","[{""description"": ""Check the audio jack and hea..."
2,"""Mouse stopped working, need replacement""","""operating_system"": ""Windows 10"", ""availab...","[{""description"": ""Restart the mouse to see if ..."
3,"""Cannot log in to SAP system.""","""operating_system"": ""Windows 10"", ""available_...","[{""description"": ""Check the SAP system for any..."
4,"""Tried to Login in SAP without success""","""operating_system"": ""Windows 10""","[{""description"": ""Check SAP system logs for an..."
...,...,...,...
198,"""Printer EB065 is printing with bad quality""","""operating_system"": ""Windows 10"", ""available_...","[{""description"": ""Check and adjust the printer..."
199,"""Cannot login in SAP""","""operating_system"": ""Windows 10"", ""availab...","[{""description"": ""Check SAP login credentials ..."
200,"""Cannot login in SAP""","""operating_system"": ""Windows 10"", ""available...","[{""description"": ""Check SAP login credentials ..."
201,"""Cannot login in SAP""","""operating_system"": ""Windows 10"", ""available...","[{""description"": ""Check SAP login credentials ..."


## Storing intermediate Dataframe locally

In [146]:
df_new.to_csv("itsm_data_atomicwork.csv",index=False)

## Referring an existing dataframe on Predibase - Used the format prompt and completion column - for instruction tuning - Mistral . For LLama the format is different

In [147]:
df_news = pd.read_csv('tldr_dataset.csv')

In [148]:
df_news

,prompt,completion,split
0,<s>[INST] The following passage is content fro...,NASA’s Ingenuity—the First Ever Off-World Heli...,train
1,<s>[INST] The following passage is content fro...,Huge space hotel promises fake gravity and 'su...,train
2,<s>[INST] The following passage is content fro...,Pomerium (GitHub Repo),train
3,<s>[INST] The following passage is content fro...,Tech Lead Expectations for Engineering Project...,train
4,<s>[INST] The following passage is content fro...,"Who was Fahim Saleh, the tech CEO brutally dis...",train
...,...,...,...
7927,<s>[INST] The following passage is content fro...,Google is letting some users test its AR navig...,evaluation
7928,<s>[INST] The following passage is content fro...,Quips is an AI to help ALS patients speak with...,evaluation
7929,<s>[INST] The following passage is content fro...,Scrollex (GitHub Repo),evaluation
7930,<s>[INST] The following passage is content fro...,"Cana Unveils Molecular Beverage Printer, a ‘Ne...",evaluation


In [135]:
print(df_news.iloc[0,0])


<s>[INST] The following passage is content from a news report. Please summarize this passage in one sentence or less.

Passage: NASA's Perseverance rover will be carrying a four-pound helicopter in its belly. Named Ingenuity, it will attempt up to five powered flights on Mars. The first flight will replicate test flights previously conducted on Earth. After that, Ingenuity will start testing its limits, eventually flying up to 150 feet away on its final test. Each trip will last about 90 seconds from takeoff to landing, which is the maximum time available due to Ingenuity's battery capacity. Mars' atmosphere is less than 1 percent the density of Earth's atmosphere, so Ingenuity's blades have to spin 10 times faster than helicopters on Earth to create an upward lift. It will take a whole Martian day to recharge between flights.

Summary: 
 [/INST] 


## Prompt structure for LLama 2 - our use case

In [147]:
PROMPT_STRUCTURE_BEGIN = """<s>[INST] You are an AI-powered troubleshooting agent tailored for IT service management (ITSM), capable of autonomously performing troubleshooting
tasks using integrated IT tools and systems.<</SYS>> The example below highlight an IT issue with information about Operating system used and avaiable tools, the issues range from
network problems to software malfunctions etc., 

Issue: The main office printer is not responding to print commands. The operating system used is Windows 10 and available tools: ["Printer Management Software", "Print Job Management Tool", "Hardware Diagnostic Tools", "Atomicwork"]
            
Resolution:  "tasks": [
            {"description": "Check the printer's online status and attempt a remote reset to resolve common errors.",
                "tools": ["Printer Management Software"],
            },
            {"description": "Automatically clear the print queue if a jam is detected and restart the printing process.",
                "tools": ["Print Job Management Tool"],
            },
            {"description": "Run a diagnostic to check for hardware faults and automatically send a report to the maintenance team if necessary.",
                "tools": ["Hardware Diagnostic Tools"],
            },
            {"description": "Notify the administrative staff through the IT management system if the printer needs physical inspection.",
                "tools": ["Atomicwork"]
            },
        ]
"""
PROMPT_STRUCTURE_END = """
Please provide resolution to the issue with a specified sequence of tasks to perform and tools to be used.

Issue: {issue}

Resolution: 

 [/INST]"""


## More refined prompt - to enforce JSON output - Being used in predibase UI

In [ ]:
"""<s>[INST] You are an AI-powered troubleshooting agent tailored for IT service management (ITSM), capable of autonomously performing troubleshooting
tasks using integrated IT tools and systems.<</SYS>> The example below highlight an IT issue with information about Operating system used and avaiable tools, the issues range from
network problems to software malfunctions etc., 

Generate a structure JSON output (Resolution) with the following fields :

"tasks" : [ {"description": "",
                "tools": [""],
            },]

Example:

Issue: The main office printer is not responding to print commands. The operating system used is Windows 10 and available tools: ["Printer Management Software", "Print Job Management Tool", "Hardware Diagnostic Tools", "Atomicwork"]
            
Resolution:  "tasks": [
            {"description": "Check the printer's online status and attempt a remote reset to resolve common errors.",
                "tools": ["Printer Management Software"],
            },
            {"description": "Automatically clear the print queue if a jam is detected and restart the printing process.",
                "tools": ["Print Job Management Tool"],
            },
            {"description": "Run a diagnostic to check for hardware faults and automatically send a report to the maintenance team if necessary.",
                "tools": ["Hardware Diagnostic Tools"],
            },
            {"description": "Notify the administrative staff through the IT management system if the printer needs physical inspection.",
                "tools": ["Atomicwork"]
            },
        ]


Please provide resolution to the issue with a specified sequence of tasks to perform and tools to be used and use the JSON output structure strictly as shown above in the example. 
Issue: The laptop is not working.

Resolution: 

 [/INST]"""


In [148]:
PROMPT_STRUCTURE_BEGIN = PROMPT_STRUCTURE_BEGIN.replace("\n"," ")
PROMPT_STRUCTURE_BEGIN = PROMPT_STRUCTURE_BEGIN.replace("  "," ")

## creating prompt and completion column data for final dataframe - to be used for instructional tuning of LLama2

In [149]:
def get_tool_os(metadata):
    
    import json

    # Given string
    input_str = metadata

    # Add braces to make it a valid JSON string
    json_str = '{' + input_str.rstrip(',') + '}'

    # Parse the JSON string
    data = json.loads(json_str)

    # Extract the desired values
    operating_system = data["operating_system"]
    if "available_tools" in data:   
        available_tools = data["available_tools"]
         # Generate the output string
        output_str = f'. The operating system used is {operating_system} and available tools: {available_tools}'
    else:
         output_str = f'. The operating system used is {operating_system} and available tools: Unknown'

    return output_str

In [268]:
# input_str = df_new.iloc[200,1]

# # Add braces to make it a valid JSON string
# json_str = '{' + input_str.rstrip(',') + '}'
# json_str = json_str.replace("  ","")
# # # Parse the JSON string
# data = json.loads(json_str)

In [150]:
dic_data_model = {"prompt":[],"completion":[]}
for index, row in df_new.iterrows():
    context = row['Context']
    metadata = get_tool_os(row['Metadata'])
    dic_data_model['prompt'].append(PROMPT_STRUCTURE_BEGIN+PROMPT_STRUCTURE_END.format(issue=context+metadata))
    dic_data_model['completion'].append(row['Tasks'])

In [151]:
df_final = pd.DataFrame(dic_data_model)

In [152]:
df_final

,prompt,completion
0,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check for any paper jams or ..."
1,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check the audio jack and hea..."
2,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Restart the mouse to see if ..."
3,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check the SAP system for any..."
4,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check SAP system logs for an..."
...,...,...
198,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check and adjust the printer..."
199,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check SAP login credentials ..."
200,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check SAP login credentials ..."
201,<s>[INST] You are an AI-powered troubleshootin...,"[{""description"": ""Check SAP login credentials ..."


In [153]:
df_final["completion"][0]

'[{"description": "Check for any paper jams or misaligned printing and restart the print job.", "tools": ["Printer Management Software", "Atomicwork"]}, {"description": "Configure the document properties to ensure the correct print settings are selected.", "tools": ["Printer Management Software"]}, {"description": "Verify that the printer is properly connected to the network and is receiving the print job.", "tools": ["Print Job Management Tool", "Atomicwork"]}, {"description": "If the issue persists, raise a ticket with the IT team to investigate and resolve the issue.", "tools": ["Atomicwork"]}]'

In [284]:
df_final.to_csv("final_data_itsm.csv",index=False)

## Reference 

In [ ]:

# {"prompt": ..., "completion": ...}
# {"prompt": "Please summarize the following article ...", "completion": "Madonna kicks off Celebration World Tour in London"}
# {"prompt": "Please summarize the following article ...", "completion": "Facebook Releases First Transparency Report on what Americans see on the platform"}
# {"prompt": ..., "completion": ...}

In [1]:
#https://kaggle.com/datasets/macimottin/it-incidents/code

In [ ]:
# context_pattern = r'"context": "(.*?)"'
# ": Matches the double quote character literally.
# context: Matches the word "context" literally.
# : : Matches the colon and space characters literally.
# ": Matches the double quote character literally.
# (.*?): This part captures the content within the double quotes after "context".
# ( ): Represents a capturing group.
# .*?: Matches any character (except for line terminators) lazily, meaning it will match as few characters as possible.
# ": Matches the closing double quote character literally.


In [ ]:
# Metadata Pattern:
# "metadata": {: Matches the literal string "metadata": {.
# (.*?): This part captures the content within the curly braces after "metadata".
# }: Matches the closing curly brace character literally.

In [ ]:
#Tasks pattern
# "tasks": \[: Matches the literal string "tasks": [ where \ is used to escape the [.
# (.*?): This part captures the content within the square brackets after "tasks".
# \]: Matches the closing square bracket character literally.